# Web API

---

## Methods

### GET Resources

* `GET /resources` - Some paginated list of stuff, in some logical default order for that data.
* `GET /resources/X` - Just entity X. That can be an ID, hash, slug, username, etc as long as it unique to one "resource".
* `GET /resources/X,Y,Z` - The client wants multiple things, so give them multiple things.

Sub-resources will be:

* `GET /users/X/checkins` - Find all the checkins for a specific user.
* `GET /users/X/checkins/Y` - Find a specific checkin for a specific user.

### DELETE Resources

* `DELETE /places/X` - Delete a single place.
* `DELETE /places/X,Y,Z` - Delete a bunch of places.
* `DELETE /places/X/image` - Delete the image for a place, or:
* `DELETE /places/X/images` - Delete all images.

### POST vs PUT
*PUT* is used if the entire URL is known and the action is idempotent. Idempotent is a fancy word for "can do it over and over again without causing different results".

```
PUT /places/1/image HTTP/1.1
Host: example.com
Content-Type: image/jpeg
```

There is only one image and a new attempt is an override then `PUT /places/X/image` would be ideal. But if there are multiple images, then `POST /places/X/images` could be used.

Another example couble be user settings:

* `POST /me/settings` - Only specific fields are POSTed.
* `PUT /me/settings` - All fields are sent.

## Content Structure

### JSON API

JSON API suggests that both single resources and resource collections should both be inside a plural key.
```
{
    "posts": [{
        "id": "1",
        "title": "Rails is Omakase"
    }]
}
```

### Twitter-style

Ask for one user get one user:
```
{
    "name": "Phil Sturgeon",
    "id": "13343"
}
```

Ask for a collection of things and get a collection of things:
```
[
    {
        "name": "Hulk Hogan",
        "id": "2341234"
    },
    {
        "name": "Mick Foley",
        "id": "39898234"
    }
]

### Facebook-style

Ask for one user get one user:
```
{
    "name": "Phil Sturgeon",
    "id": "13343"
}
```

Ask for a collection of things and get a collection of things, but namespaced:
```
[
    "data": [{
        "name": "Hulk Hogan",
        "id": "2341234"
    },
    {
        "name": "Mick Foley",
        "id": "39898234"
    }]
]

### Much Namespace, Nice Output

Namespace the single item.
```
{
    "data": {
        "name": "Phil Sturgeon",
        "id": "13343"
    }
}
```

Namespace the multiple items.
```
[
    "data": [{
        "name": "Hulk Hogan",
        "id": "2341234"
    },
    {
        "name": "Mick Foley",
        "id": "39898234"
    }]
]
```

## Errors

```
{
    "error": {
        "type": "OAuthException",
        "code": "ERR-01234",
        "message": "Session has expired at unix time 1234323. Current unix time is 1323432.",
        "documentation_url": "http://example.com/docs/errors/#ERR-01234"
    }
}
```

## Data Relations

### Sub-Resources

```
/places/X/checkins
```

The downside is that it is an extra HTTP request. This is know as *1 + n*, meaning the work done is increased by an extra one request for each *place* you look up.

### Foreign Key Arrays

```
{
    "post": {
        "id": 1,
        "title": "Progressive Enhancement is Dead",
        "comments": [1, 2],
        "links": {
            "user": "/people/tomdale"
        }
    }
}
```

The downside is that the API consumer has to stitch all of that data together, which could be a lot of work for a large dataset.

### Compund Documents (a.k.a Side-Loading)

```
{
    "meta": {"primaryCollection": "comments"},
    "comments": [...],
    "authors: [...]
}
```

The pro is if an embedded piece of data is commonly recurring, the same resource need not to be downloaded multiple times. The con is that context gets lost in larger data structures and it has the same issue as the "Foreign Key Array": the mapping of data to create an accurate structure is left to the API consumer.

### Embedded Documents (a.k.a Nesting)

If an API cousumer were to call the URL `/places?embed=checkins,merchant` then checkin and merchant data would be in response:
```
{
    "data": [{
        "id": 2,
        "name": "Videology",
        "lat": 40.713857,
        "lon": -73.961936,
        "created_at": "2013-04-02",
        "checkins" : [
            // ...
        ],
        "merchant" : {
            // ...
        }
    },
    {
        "id": 1,
        "name": "Barcade",
        "lat": 40.712017,
        "lon": -73.950995,
        "created_at": "2012-09-23",
        "checkins" : [
            // ...
        ],
        "merchant" : {
            // ...
        }
    }]
}
```

## Pagination

```
/places?number=12

{
    "data": [
        ...
    ],
    "paginatin": {
        "total": 1000,
        "count": 12,
        "per_page": 12,
        "current_page": 1,
        "total_pages": 84,
        "next_url": "https://api.example.com/places?page=2&number=12"
    }
}
```